# LDA for TimeSets

*Apply LDA to find topics for the TimeSets paper.*

In [1]:
import json

from gensim.models.ldamodel import LdaModel
from gensim import corpora

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

from collections import Counter
from itertools import chain

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

## Data Preprocessing

#### Load the data

In [2]:
def prepare_documents_facebook(filename):
    'Return a list of documents ready for topic modelling.'
    with open(filename) as f:
        posts = json.load(f)
        return [post['message'] for post in posts if 'message' in post]

In [3]:
data = prepare_documents_facebook('../data/facebook.json')

#### Clean the data

In [4]:
def preprocess_documents(docs):
    'Return a bag-of-word representation of the documents after cleaning (stopwords/punctuation removal, stemming).'
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation + '“”’—') 
    lemma = WordNetLemmatizer()
    return [clean(doc, stop, exclude, lemma) for doc in docs]

def clean(doc, stop, exclude, lemma):
    punc_free_doc = ''.join(c for c in doc if c not in exclude)
    stop_free_words = [w for w in punc_free_doc.lower().split() if w not in stop]    
    normalized_words = [lemma.lemmatize(w) for w in stop_free_words if len(w) >= 3]
    return normalized_words

In [5]:
docs = preprocess_documents(data)
for i in range(0, 10):
    print(data[i], docs[i])
    print()

The Syrian military declared today that the U.S.-Russia brokered cease-fire is over, blaming rebel groups for violating the agreement. ['syrian', 'military', 'declared', 'today', 'usrussia', 'brokered', 'ceasefire', 'blaming', 'rebel', 'group', 'violating', 'agreement']

Rose Pak, an influential community activist who turned San Francisco's Asian-American population into a political power in the city, passes away at 68. ['rose', 'pak', 'influential', 'community', 'activist', 'turned', 'san', 'franciscos', 'asianamerican', 'population', 'political', 'power', 'city', 'pass', 'away']

Warplanes target the besieged Syrian city of Aleppo for the first time since the ceasefire went into effect last week. ['warplane', 'target', 'besieged', 'syrian', 'city', 'aleppo', 'first', 'time', 'since', 'ceasefire', 'went', 'effect', 'last', 'week']

Using Skittles to make a point about “our Syrian refugee problem” didn’t go over too well with Mars, Incorporated—but the candy maker’s rebuke of Donald Tr

#### Look at word frequency

In [6]:
word_dict = Counter(chain.from_iterable(docs))
word_counts = sorted(word_dict.items(), key=lambda x: -x[1])
word_counts[:10]

[('trump', 204),
 ('donald', 165),
 ('clinton', 110),
 ('hillary', 95),
 ('say', 80),
 ('president', 74),
 ('debate', 63),
 ('presidential', 51),
 ('obama', 47),
 ('first', 40)]

## Topic Modelling with LDA

#### Find topics of the whole corpus

In [7]:
def build_lda(corpus, num_topics=10, passes=10, alpha='symmetric', eta=None):
    'Return an LDA model from the given doc-term matrix .'
    return LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes, alpha=alpha, eta=eta, random_state=0)

def get_model_topics(lda):
    return [[(lda.id2word[t], '{:.3f}'.format(p)) for t, p in lda.get_topic_terms(i)] for i in range(lda.num_topics)]

In [ ]:
dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [19]:
lda = build_lda(corpus, num_topics=10)
get_model_topics(lda)

[[('trump', '0.030'),
  ('donald', '0.020'),
  ('debate', '0.015'),
  ('presidential', '0.011'),
  ('republican', '0.010'),
  ('clinton', '0.009'),
  ('hillary', '0.009'),
  ('first', '0.007'),
  ('cnn', '0.006'),
  ('country', '0.006')],
 [('trump', '0.014'),
  ('donald', '0.013'),
  ('say', '0.012'),
  ('presidential', '0.009'),
  ('week', '0.007'),
  ('news', '0.007'),
  ('abc', '0.007'),
  ('republican', '0.007'),
  ('hillary', '0.006'),
  ('clinton', '0.006')],
 [('trump', '0.028'),
  ('president', '0.020'),
  ('donald', '0.019'),
  ('debate', '0.012'),
  ('clinton', '0.011'),
  ('obama', '0.010'),
  ('hillary', '0.010'),
  ('say', '0.008'),
  ('week', '0.006'),
  ('republican', '0.006')],
 [('trump', '0.024'),
  ('donald', '0.018'),
  ('like', '0.013'),
  ('democrat', '0.009'),
  ('occupy', '0.008'),
  ('page', '0.008'),
  ('first', '0.008'),
  ('get', '0.008'),
  ('say', '0.007'),
  ('new', '0.007')],
 [('donald', '0.022'),
  ('trump', '0.022'),
  ('clinton', '0.017'),
  ('hilla

Couldn't tell what topics are about! Is it because the term probability is not a good measure here? Let's use pyLDAvis to look at other measures.

In [20]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      16.568277        1       1 -0.125381  0.012253
2      11.823414        1       2 -0.051177 -0.011642
3      11.459519        1       3 -0.004667 -0.139606
4      10.725240        1       4 -0.038576 -0.048071
1      10.197028        1       5  0.035281 -0.007013
0       9.223119        1       6 -0.005915  0.102139
5       8.999933        1       7 -0.073269  0.058488
8       7.902038        1       8  0.053706 -0.023221
6       7.137116        1       9  0.123687 -0.001953
7       5.964325        1      10  0.086312  0.058626, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
476   Default   83.000000       clinton   83.000000  30.0000  30.0000
482   Default   71.000000       hillary   71.000000  29.0000  29.0000
223   Default   18.000000         would   18.000000  28.0000  28.0000
403   Default   55.000000     president   55.000000  27.0000  27.0000
892   Default   17.000000          like   17.000000  26.0000  26.0000
756   Default    8.000000          need    8.000000  25.0000  25.0000
1248  Default    7.000000        occupy    7.000000  24.0000  24.0000
36    Default   13.000000          week   13.000000  23.0000  23.0000
245   Default   35.000000         obama   35.000000  22.0000  22.0000
527   Default   47.000000        debate   47.000000  21.0000  21.0000
342   Default   13.000000      democrat   13.000000  20.0000  20.0000
132   Default   26.000000      campaign   26.000000  19.0000  19.0000
717   Default   26.000000    republican   26.000000  18.0000  18.0000
139   Default   18.000000      election   18.000000  17.0000  17.0000
41    Default  124.000000        donald  124.000000  16.0000  16.0000
418   Default   10.000000          page   10.000000  15.0000  15.0000
149   Default   12.000000           abc   12.000000  14.0000  14.0000
121   Default    8.000000         leave    8.000000  13.0000  13.0000
200   Default    7.000000          2016    7.000000  12.0000  12.0000
1979  Default    4.000000       excited    4.000000  11.0000  11.0000
227   Default   37.000000  presidential   37.000000  10.0000  10.0000
96    Default    7.000000           man    7.000000   9.0000   9.0000
157   Default   29.000000           new   29.000000   8.0000   8.0000
97    Default    5.000000        person    5.000000   7.0000   7.0000
500   Default    8.000000           win    8.000000   6.0000   6.0000
474   Default   16.000000     candidate   16.000000   5.0000   5.0000
1445  Default    8.000000          ever    8.000000   4.0000   4.0000
566   Default    6.000000         image    6.000000   3.0000   3.0000
98    Default   59.000000           say   59.000000   2.0000   2.0000
1534  Default    9.000000           cnn    9.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
966   Topic10    1.308022       backing    3.423810   1.8571  -5.7576
1701  Topic10    1.308015       priebus    3.434423   1.8540  -5.7576
1702  Topic10    1.308015        reince    3.434423   1.8540  -5.7576
189   Topic10    1.308019   negotiation    3.463929   1.8455  -5.7576
1991  Topic10    1.308026       capitol    3.475727   1.8421  -5.7576
1992  Topic10    1.308026          hill    3.475728   1.8421  -5.7576
170   Topic10    1.930898      official    6.434039   1.6158  -5.3681
81    Topic10    1.308017         mylan    3.574136   1.8142  -5.7576
192   Topic10    1.254941       primary    3.519846   1.7880  -5.7990
313   Topic10    1.308024         chair    4.031842   1.6937  -5.7576
1703  Topic10    1.308014           rnc    4.170090   1.6599  -5.7576
1238  Topic10    1.305385           got    4.824130   1.5122  -5.7596
61    Topic10    1.305052          case    5.029452   1.4703  -5.7599
161   Topic10    1.803145     according   10.647762   1.0436  -5.4366
717   Topic10    2.

What is this dataset about? Should I exlude high-frequency terms?

#### Filter out high-frequency terms

#### Get topics associated with documents

In [74]:
def get_topics_for_documents(lda, corpus):
    'Return top 5 topic probabilities for each document in the given corpus.'
    return [get_topics_for_one_document(lda, doc) for doc in corpus]

def get_topics_for_one_document(lda, doc):
    topics = sorted(lda.get_document_topics(doc, minimum_probability=10**-6), key = lambda x: -x[1])[:5]
    return [float('{:.3f}'.format(p)) for (t, p) in topics]

In [75]:
output_topics = get_topics_for_documents(lda, corpus)
output_topics[:10]

[[0.931, 0.008, 0.008, 0.008, 0.008],
 [0.944, 0.006, 0.006, 0.006, 0.006],
 [0.94, 0.007, 0.007, 0.007, 0.007],
 [0.959, 0.005, 0.005, 0.005, 0.005],
 [0.975, 0.003, 0.003, 0.003, 0.003],
 [0.925, 0.008, 0.008, 0.008, 0.008],
 [0.936, 0.007, 0.007, 0.007, 0.007],
 [0.95, 0.006, 0.006, 0.006, 0.006],
 [0.961, 0.004, 0.004, 0.004, 0.004],
 [0.94, 0.007, 0.007, 0.007, 0.007]]

#### Find terms associated with topics 

In [76]:
def get_terms_for_topics(lda):
    'Return term probabilities for each topic. Only top 5 terms.'
    return [get_terms_for_one_topic(lda, i) for i in range(lda.num_topics)]

def get_terms_for_one_topic(lda, topic_id):
    return [float('{:.3f}'.format(p)) for t, p in lda.get_topic_terms(topic_id, topn=5)]

In [77]:
output_terms = get_terms_for_topics(lda)
output_terms

[[0.042, 0.036, 0.025, 0.021, 0.014],
 [0.015, 0.013, 0.012, 0.011, 0.007],
 [0.016, 0.011, 0.008, 0.008, 0.008],
 [0.019, 0.018, 0.014, 0.01, 0.008],
 [0.016, 0.016, 0.008, 0.008, 0.007],
 [0.022, 0.017, 0.013, 0.01, 0.01],
 [0.018, 0.013, 0.013, 0.01, 0.009],
 [0.013, 0.009, 0.008, 0.008, 0.007],
 [0.021, 0.014, 0.014, 0.014, 0.012],
 [0.029, 0.028, 0.028, 0.023, 0.013]]

## Export Model Data

In [78]:
def export_model_data(lda, corpus):
    'Return a dictionary detailing model parameters and probability matrices.'
    return {
        'alpha': float(lda.alpha[0]), # assume the same value for all elements
        'beta': float(lda.eta[0]), # assume the same value for all elements
        'num_topics': lda.num_topics,
        'doc_topics': get_topics_for_documents(lda, corpus),
        'topic_terms': get_terms_for_topics(lda)
    }

In [79]:
def save_file(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)

#### Different alpha values

In [80]:
def export_data_for_alphas(corpus, alphas, filename):
    data = [export_model_data(build_lda(corpus, alpha=alpha), corpus) for alpha in alphas]
    save_file(data, filename)

In [81]:
export_data_for_alphas(corpus, [0.01, 0.03, 0.1, 0.3, 1, 3, 10], '../data/facebook-alphas.json')